In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit

In [6]:
%run ../classifier/report.ipynb

In [8]:
default_params = {
    "activation": "relu",
    "alpha": 0.0001,
    "beta_1": 0.9,
    "beta_2": 0.999,
    "hidden_layer_sizes": (100,),
    "max_iter": 200,
    "n_iter_no_change": 10,
    "validation_fraction": 0.1,
    "warm_start": False,
}


def get_model(params=default_params):
    return MLPClassifier(**params)


def classifier(base, train_size=False, params={}):
    model = get_model(params)

    if train_size != False:
        cross_validation = StratifiedShuffleSplit(n_splits=10, train_size=train_size)
    else:
        cross_validation = StratifiedKFold(n_splits=10, shuffle=True)

    report = Report()

    y = base[:, -1]
    X = base[:, 0:-1]

    for train_index, test_index in cross_validation.split(X, y):
        x_train_fold, x_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        model.fit(x_train_fold, y_train_fold)
        y_pred = model.predict(x_test_fold)
        report.register_report_fold(y=y_test_fold, y_pred=y_pred)

    print(report.report_folds[-1])
    report.print_report()
    return model, report